In [ ]:
import panel as pn
from ipyleaflet import Map, basemaps, DrawControl, AwesomeIcon, Marker, GeoData, GeoJSON, LayersControl, Popup, FullScreenControl, CircleMarker, MarkerCluster, LegendControl
from ipywidgets import Layout, HTML
from datetime import date
import os
import pandas as pd
from io import StringIO
import hvplot.pandas
from bokeh.sampledata.degrees import data as degrees
from collections import defaultdict
import geopandas, json
from bokeh.sampledata.sample_geojson import geojson as nhs_feature_collection
import math
pn.extension("ipywidgets", sizing_mode="stretch_width")

# Specify index of the starting/active tab that displays on load.
tabs = pn.Tabs(active=6, tabs_location="left")

In [ ]:
## Create widgets.
headings = pn.pane.Markdown('''
## Approximately how many seconds have you lived?
### Enter your information here and see!
''')
name = pn.widgets.TextInput(name="Name", value="Venus")
age = pn.widgets.IntSlider(name="Age", value=50, start=1, end=100)
color = pn.widgets.ColorPicker(name="Color", value="#4f4fdf")
calc_button = pn.widgets.Button(name="Calculate", button_type="primary")
result = pn.widgets.StaticText()

## Create functions.
def compute_secs_lived(event):
  secs = age.value * 365 * 24 * 60 * 60
  result.value = f"Hello {name.value}, you have lived for approximately {secs} seconds!"
  result.style = {"color": color.value}
  return result

## Create interactions between widgets and functions.
calc_button.on_click(compute_secs_lived)

## Create person component and add it as a tab.
person_inputs = pn.Row(name, age, color)
person_component = pn.Column(headings, person_inputs, calc_button, result)
tabs.append(("Person", person_component))

In [ ]:
## Create map widgets.
center = [37, -122]     # center = [vertical distance on y-axis, horizontal distance on x-axis] -> [0,0] is underneath Ghana, Africa in South Atlantic Ocean
map = Map(basemap=basemaps.OpenStreetMap.France, center=center, zoom=10, layout=Layout(height="75vh"))
icon = AwesomeIcon(
  name="heart",
  marker_color="blue",
  icon_color="plum",
  spin=True
)
marker = Marker(icon=icon, location=(center[0] - 0.035, center[1] - 0.015), title="Santa Cruz Beach Boardwalk")
map.add_layer(marker)

display_geo_button = pn.widgets.Button(name="Display Geometry", button_type="success")
clear_drawings_button = pn.widgets.Button(name="Clear Drawings")

geo = ""
drawing_geometry = pn.widgets.StaticText(name="Last Drawing's Geometry", value=geo)
count = 0
num_drawings = pn.widgets.StaticText(name="Number of Drawings", value=count)

## Create and customize drawing controls.
draw_control = DrawControl(polyline={}, polygon={}, circle={}, circlemarker={})
draw_control.rectangle = {
  "shapeOptions": {
    "fillColor": "#4f4fdf",
    "color": "#4f4fdf",
    "fillOpacity": 0.3
  }
}
map.add_control(draw_control)

# Save drawn rectangle and increment number of drawings.
def save_drawing(self, action, geo_json):
  global geo, count
  geo = geo_json["geometry"]
  count += 1
  num_drawings.value = count // 2

# Display geometry of recently drawn rectangle and total number of drawings.
def print_drawing_geometry(event):
  # drawing_geometry.value = draw_control.last_draw["geometry"]
  drawing_geometry.value = geo
  

# Clear all drawings and reset drawing count.
def clear_drawings(event):
  draw_control.clear()
  global geo, count
  geo = ""
  drawing_geometry.value = geo
  count = 0
  num_drawings.value = count

## Create interactions between widgets and functions.
draw_control.on_draw(save_drawing)
display_geo_button.on_click(print_drawing_geometry)
clear_drawings_button.on_click(clear_drawings)

## Create map component and add it as a tab.
map_buttons = pn.Row(display_geo_button, clear_drawings_button)
map_component = pn.Column(map, map_buttons, drawing_geometry, num_drawings)
tabs.append(("Map", map_component))

In [ ]:
## Create CoastSeg controls with Panel widgets.
date_heading = pn.pane.Markdown('''
## Pick a date:
''')
start_date_picker = pn.widgets.DatePicker(name="Start Date", value=date(2018,12,1))
end_date_picker = pn.widgets.DatePicker(name="End Date", value=date(2019,3,1))
collection_heading = pn.pane.Markdown('''
## Pick a collection:
C01: LandSat Collection 1 (all images prior to 2022-01-01)\n
C02: LandSat Collection 2 (all images after to 2022-01-01)\n
- LandSat 9 is only available in C02
''')
collection_radio_button_group = pn.widgets.RadioButtonGroup(name="Collection", options=["C01", "C02"], button_type="primary", orientation="vertical")
# collection_radio_box_group = pn.widgets.RadioBoxGroup(name="Collection", options=["C01", "C02"], inline=False)
satellites_heading = pn.pane.Markdown('''
## Pick at least one satellite:
Unselect a satellite by clicking on the option again.
''')
satellites_check_button_group = pn.widgets.CheckButtonGroup(name="Satellites", options=["L5", "L7", "L8", "S2"], value=["L8"], button_type="warning")
# satellites_multi_select = pn.widgets.MultiSelect(name="Satellites", options=["L5", "L7", "L8", "S2"], value=["L8"])

## Create CoastSeg controls component and add it as a tab.
date_pickers = pn.Row(start_date_picker, end_date_picker)
coastseg_component = pn.Column(date_heading, date_pickers, collection_heading, collection_radio_button_group, satellites_heading, satellites_check_button_group)
tabs.append(("CoastSeg", coastseg_component))

In [ ]:
## Create data widgets.
data_files = os.listdir("data")
file_select = pn.widgets.Select(name="File", options=data_files, value=data_files[0])

# Binds file_select widget's selected value to selected_filename argument.
@pn.depends(file_select)
def get_selected_file_path(selected_filename):
  return "data/" + selected_filename

selected_file_download = pn.widgets.FileDownload(callback=get_selected_file_path, filename=file_select.value, button_type="success", align="end")

# Link value parameter of source (file_select widget) to filename parameter of target (selected_file_download button).
file_select.link(selected_file_download, value="filename")

data = pd.read_csv("data/A_naip_meta_served.csv")
data = data.set_index("label_image_filename")
years_options = list(data.year.unique())
years_multi_choice = pn.widgets.MultiChoice(name="Years", options=years_options, value=[years_options[0]], solid=False)
doodle_spatial_density_range_slider = pn.widgets.RangeSlider(name="Proportion of Image Annotated", start=data.doodle_spatial_density.min(), end=data.doodle_spatial_density.max(), bar_color="#4f4fdf")

# Filter data based on doodle_spatial_density.
@pn.depends(years_multi_choice, doodle_spatial_density_range_slider)
def get_filtered_data(years, density):
  global data
  filtered_data = data
  [min_density, max_density] = density
  if years_multi_choice.value:
    filtered_data = filtered_data[filtered_data.year.isin(years)]
  return filtered_data[(filtered_data.doodle_spatial_density >= min_density) & (filtered_data.doodle_spatial_density <= max_density)]

# Saves filtered data's DataFrame object as a CSV, and converts it into a file StringIO object.
@pn.depends(years_multi_choice, doodle_spatial_density_range_slider)
def get_filtered_file(ymc, dsdrs):
  dataframe = get_filtered_data(ymc, dsdrs)
  sio = StringIO()
  dataframe.to_csv(sio)
  sio.seek(0)
  return sio

filtered_data_file_download = pn.widgets.FileDownload(callback=get_filtered_file, filename="filtered_data.csv", button_type="primary")

## Create data component and add it as a tab.
download_selected_file = pn.Row(file_select, selected_file_download)
filter_controls = pn.Row(years_multi_choice, doodle_spatial_density_range_slider)
info_text = """
  ### Filtered Data
  The following data is a preview of the content in `filtered_data.csv`:
"""
data_component = pn.Column("## Select a File to Download", download_selected_file, "## Filter Data to Download", filter_controls, filtered_data_file_download, pn.pane.Alert(info_text, alert_type="info"), pn.panel(get_filtered_data))
tabs.append(("Data", data_component))

In [ ]:
## Create plot widgets.
preview_degrees_data = degrees.head()
degree_options = list(degrees.columns[1:])
degree_multi_choice = pn.widgets.MultiChoice(name="Degree", options=degree_options, value=[degree_options[2], degree_options[3], degree_options[6]], solid=False)

# Creates line plot for all selected degrees.
@pn.depends(degree_multi_choice)
def plot_degree(degree_vals):
  return degrees.hvplot.line(x="Year", y=degree_vals, value_label="% of Degrees Earned by Women", legend="bottom")

nhs_map = Map(center=[53, -3], zoom=6, layout=Layout(height="75vh"))

# Convert JSON data into a GeoDataFrame, which allows us to view the data in a Panel table.
nhs_geodataframe = geopandas.GeoDataFrame.from_features(json.loads(nhs_feature_collection))
preview_nhs_data = nhs_geodataframe.head()

team_info = HTML()
team_popup = Popup(location=[100, -100], child=team_info, name="Team Information")
team_popup.close_popup()    # Initially hide team info popup when the map loads.
nhs_map.add_layer(team_popup)

# Opens a popup that displays team info when clicking/hovering over its marker.
def display_team_info(feature, **kwargs):
  team_popup.location = list(reversed(feature["geometry"]["coordinates"]))
  team_info.value = """
    <b>Name</b>: {}<br>
    <b>Location</b>: {}, {}<br>
    <b>Phone</b>: {}<br>
    <b>Email</b>: {}<br>
    <b>Website</b>: <a href="{}">{}</a>
  """.format(
    feature["properties"]["OrganisationName"],
    feature["properties"]["City"],
    feature["properties"]["County"],
    feature["properties"]["Phone"],
    feature["properties"]["Email"],
    feature["properties"]["Website"],
    feature["properties"]["Website"]
  )
  team_popup.open_popup()

# Option 1: Convert GeoDataFrame into GeoData, which is a layer that can be displayed on the Map.
# nhs_geodata = GeoData(geo_dataframe=nhs_geodataframe, name="NHS Teams")
# nhs_map.add_layer(nhs_geodata)

# Option 2: Convert JSON data into GeoJSON, which is a layer that can be displayed on the Map.
nhs_geojson = GeoJSON(data=json.loads(nhs_feature_collection), hover_style={"width": "25px", "height": "25px", "borderRadius": "50%", "color": "yellow", "fillOpacity": 0.7}, name="NHS Teams")
nhs_geojson.on_click(display_team_info)
nhs_geojson.on_hover(display_team_info)
nhs_map.add_layer(nhs_geojson)

nhs_map.add_control(LayersControl(position="topright"))

## Create plots component and add it as a tab.
degree_subcomponents = pn.Column("## Bachelor's Degrees Earned by Women", preview_degrees_data, degree_multi_choice, plot_degree)
nhs_subcomponents = pn.Column("## GeoJSON Data for [United Kingdom NHS England Area Teams](https://docs.bokeh.org/en/latest/docs/reference/sampledata.html#module-bokeh.sampledata.sample_geojson)", preview_nhs_data, nhs_map)
plots_component = pn.Column(degree_subcomponents, nhs_subcomponents)
tabs.append(("Plots", plots_component))

In [ ]:
## Create customized/styled widgets.
m = Map(center=(50, 0), zoom=5)
marker1 = Marker(location=(48, -2))
marker2 = Marker(location=(50, 0))
marker3 = Marker(location=(52, 2))
cluster = MarkerCluster(
  markers=(marker1, marker2, marker3)
)
m.add_layer(cluster)

## Create custom component and add it as a tab.
custom_component = pn.Column(m)
tabs.append(("Custom", custom_component))

In [ ]:
## Create Elwha Topo-Bathy Data widgets.
elwha_map = Map(center=[48.148, -123.553], zoom=15, max_zoom=19, layout=Layout(height="100vh"))
# Assign names for map's data types.
topography_data = "Topography"
bathymetry_kayak_data = "Nearshore Bathymetry - Kayak"
bathymetry_watercraft_data = "Nearshore Bathymetry - Personal Watercraft"
grainsize_data = "Surface-Sediment Grain-Size Distributions"
# elwha_data_types = [topography_data, bathymetry_kayak_data, bathymetry_watercraft_data]      # list of directories to get map data from
elwha_data_types = [topography_data, grainsize_data]
elwha_map_layers = defaultdict(list)      # {data_type: [layer1, layer2, ...]} dictionary to store all data layers that could be on the map
elwha_legend = LegendControl(
  name="Types of Data",
  position="bottomright",
  legend={
    topography_data: "red",
    bathymetry_kayak_data: "yellow",
    bathymetry_watercraft_data: "green",
    grainsize_data: "#975411"
  }
)
elwha_map.add_control(elwha_legend)
popup_content = HTML()
elwha_popup = Popup(location=[0, 0], child=popup_content, min_width=300, max_width=500, auto_close=False, name="Popup")
elwha_map.add_layer(elwha_popup)
elwha_map.add_control(FullScreenControl())
elwha_map.add_control(LayersControl(position="topright"))
elwha_data_multi_choice = pn.widgets.MultiChoice(name="Type of Data", value=[grainsize_data], options=elwha_data_types, solid=False)

# Get the latitude column of a dataframe.
def get_latitude(dataframe):
  if "Latitude" in dataframe: return dataframe["Latitude"]      # for topography and some of grainsize data
  elif "latitude" in dataframe: return dataframe["latitude"]      # for bathymetry data
  elif "Latitude (deg. N)" in dataframe: return dataframe["Latitude (deg. N)"]      # for most of grainsize data
  else: return ""

# Get the longitude column of a dataframe.
def get_longitude(dataframe):
  if "Longitude" in dataframe: return dataframe["Longitude"]      # for topography and some of grainsize data
  elif "longitude" in dataframe: return dataframe["longitude"]      # for bathymetry data
  elif "Longitude (deg. E)" in dataframe: return dataframe["Longitude (deg. E)"]      # for most of grainsize data
  else: return ""

# Gets the time that the data was collected.
def get_time_collected(data):
  if "Time (GMT)" in data: return data["Time (GMT)"]      # for grainsize data before July 2018
  elif "Time_GMT" in data: return data["Time_GMT"]      # for grainsize data at and after July 2018
  elif "Survey_Date" in data: return data["Survey_Date"]      # for topo-bathy data before July 2018
  elif "datetime_utc" in data: return data["datetime_utc"] + " UTC"      # for topo-bathy data at and after July 2018
  else: return ""

# Gets the orthometric height from the given data.
def get_ortho_height(data):
  if "Ortho_Ht_m" in data: return data["Ortho_Ht_m"]
  elif "Ortho_ht_m" in data: return data["Ortho_ht_m"]
  elif "ortho_ht_m" in data: return data["ortho_ht_m"]
  else: return ""

# Displays information for the hovered/clicked grain datapoint in a popup.
def display_grain_info(feature, **kwargs):
  elwha_popup.location = list(reversed(feature["geometry"]["coordinates"]))
  grain_properties = feature["properties"]
  popup_content.value = """
    <b>Date & Time Collected</b> {} {} GMT<br>
    <b>Sample Type</b> {}<br>
    <b>Weight</b> {}%<br>
    <b>Gravel</b> {}%<br>
    <b>Sand</b> {}%<br>
    <b>Silt</b> {}%<br>
    <b>Clay</b> {}%<br>
    <b>Mud</b> {}%<br>
  """.format(
    grain_properties["Date Collected"],
    get_time_collected(grain_properties),
    grain_properties["Sample Type"],
    grain_properties["Wt. percent in -2.00 phi bin"],
    grain_properties["Percent Gravel"],
    grain_properties["Percent Sand"],
    grain_properties["Percent Silt"],
    grain_properties["Percent Clay"],
    grain_properties["Percent Mud"]
  )
  elwha_popup.open_popup(location=elwha_popup.location)

# Displays information for the hovered/clicked topo-bathy datapoint in a popup.
def display_topo_bathy_info(feature, **kwargs):
  elwha_popup.location = list(reversed(feature["geometry"]["coordinates"]))
  topo_bathy_properties = feature["properties"]
  popup_content.value = """
    <b>Date & Time Collected</b> {}<br>
    <b>Orthometric Height</b> {} meters<br>
  """.format(
    get_time_collected(topo_bathy_properties),
    get_ortho_height(topo_bathy_properties)
  )
  elwha_popup.open_popup(location=elwha_popup.location)

# Get the point_style based on the data type.
def get_data_type_point_style(data_type):
  if data_type == topography_data:
    return {"color": "red", "opacity": 0.5, "radius": 0}     # simple dot for topography data since there's many datapoints for one latitude-longitude coordinate
  elif data_type == bathymetry_kayak_data:
    return {"color": "yellow", "radius": 0}
  elif data_type == bathymetry_watercraft_data:
    return {"color": "green", "radius": 0}
  elif data_type == grainsize_data:
    return {"color": "#975411", "opacity": 0.5, "fillColor": "#975411", "fillOpacity": 0.3, "radius": 8, "weight": 1, "dashArray": 2}
  return {}

# Get the hover_style based on the data type.
def get_data_type_hover_style(data_type):
  if data_type == topography_data:
    return {"color": "#0066cc"}
  return {"color": "#0066cc", "fillColor": "#0066cc", "weight": 3}

## Filter data based on what user wants to see.
def filter_data_on_map(event):
  selected_data_types = elwha_data_multi_choice.value
  for data_type in elwha_data_types:
    for layer in elwha_map_layers[data_type]:
      # Add the data layer if it isn't in map yet.
      if (data_type in selected_data_types) and (layer not in elwha_map.layers):
        print("display", layer.name)
        elwha_map.layers += (layer,)
        # elwha_map.add_layer(layer)
        
        # # Add mouse event handlers.
        # if data_type == grainsize_data:
        #   layer.on_click(display_grain_info)
        #   layer.on_hover(display_grain_info)
        # elif (data_type == topography_data) or (data_type == bathymetry_kayak_data) or (data_type == bathymetry_watercraft_data):
        #   layer.on_click(display_topo_bathy_info)
        #   layer.on_hover(display_topo_bathy_info)

        # # Group data from the same location into clusters.
        # marker_cluster, cluster_location = MarkerCluster(name="Clusters"), None
        # for (index, row) in dataframe.iterrows():
        #   marker_lat, marker_long = get_latitude(row), get_longitude(row)
        #   cluster_lat, cluster_long = round(marker_lat, 4), round(marker_long, 4)
        #   marker = Marker(location=[marker_lat, marker_long], visible=False)
        #   # Add new marker cluster to map when the coordinates don't belong to the current cluster.
        #   if (cluster_location is not None) and (not math.isclose(cluster_lat, cluster_location[0]) or not math.isclose(cluster_long, cluster_location[1])):
        #     # print("new cluster at " + str(cluster_location) + " is added to the map")
        #     elwha_map.add_layer(marker_cluster)
        #     marker_cluster, cluster_location = MarkerCluster(name="Clusters"), [cluster_lat, cluster_long]
        #   # Else add marker to the current cluster.
        #   else:
        #     if cluster_location is None: cluster_location = [cluster_lat, cluster_long]
        #     # print("marker at [" + str(marker_lat) + ", " + str(marker_long) + "] is added to the current cluster at " + str(cluster_location))
        #     marker_cluster.markers += (marker,)
        # elwha_map.add_layer(marker_cluster)     # need to add last marker cluster because last marker/row in dataframe will be added to last cluster but for loop never adds the cluster to the map
      
      # Else remove the data layer if user didn't select to display it and its data is in the map.
      elif (data_type not in selected_data_types) and (layer in elwha_map.layers):
        print("remove", layer.name)
        new_displayed_layers = list(elwha_map.layers)
        new_displayed_layers.remove(layer)
        elwha_map.layers = tuple(new_displayed_layers)
        # elwha_map.remove_layer(layer)
  # return elwha_map

elwha_data_multi_choice.param.watch(filter_data_on_map, "value")

## Plot collected data as GeoData layers.
for data_type in elwha_data_types:
  data_files = os.listdir("data/Elwha/" + data_type)
  for file in data_files:
    # Convert each data file into GeoData.
    print("Loading data from " + file + "...")
    dataframe = pd.read_csv("data/Elwha/" + data_type + "/" + file)
    geodataframe = geopandas.GeoDataFrame(dataframe, geometry=geopandas.points_from_xy(get_longitude(dataframe), get_latitude(dataframe)))
    # geodata = GeoData(
    #   geo_dataframe = geodataframe,
    #   point_style=get_data_type_point_style(data_type),
    #   hover_style=get_data_type_hover_style(data_type),
    #   name = file
    # )
    # # Save GeoData layer.
    # elwha_map_layers[data_type].append(geodata)
    geojson_str = geodataframe.to_json()
    geojson = GeoJSON(
      data=json.loads(geojson_str),
      point_style=get_data_type_point_style(data_type),
      hover_style=get_data_type_hover_style(data_type),
      name = file
    )
    # Add mouse event handlers.
    if data_type == grainsize_data:
      geojson.on_click(display_grain_info)
      geojson.on_hover(display_grain_info)
    elif (data_type == topography_data) or (data_type == bathymetry_kayak_data) or (data_type == bathymetry_watercraft_data):
      geojson.on_click(display_topo_bathy_info)
      geojson.on_hover(display_topo_bathy_info)
    # Add GeoJSON layer to map and save it.
    elwha_map.add_layer(geojson)
    elwha_map_layers[data_type].append(geojson)

## Create Elwha component and add it as a tab.
elwha_component = pn.Column("## Elwha River Delta Data", elwha_map, elwha_data_multi_choice)
tabs.append(("Elwha", elwha_component))

In [ ]:
## Add components to serve on app.
app = pn.Column("# My First Panel App!", tabs)
app.servable()